In [1]:
!nvidia-smi

Thu Nov 28 03:44:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           Off | 00000000:03:00.0 Off |                    0 |
| N/A   36C    P0              39W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pwd

/home/anhhn/training_qwen


In [4]:
!pip install scikit-learn

In [5]:
!pip install llama-cpp-python

# training

In [4]:
!python train_v1.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.7: Fast Qwen2 patching. Transformers = 4.46.1.
   \\   /|    GPU: Tesla V100-SXM2-32GB. Max memory: 31.739 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Loading checkpoint shards:   0%|                          | 0/2 [00:00<?, ?it/s]^C


In [3]:
import pandas as pd
from vllm import LLM, SamplingParams
from vllm.sampling_params import GuidedDecodingParams
from dataset import Prompter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

prompter = Prompter()

In [2]:
llm = LLM(
        model="checkpoint/unsloth.Q5_K_M.gguf", max_model_len=4096, gpu_memory_utilization=0.4
    )

INFO 11-20 10:07:22 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 11-20 10:07:27 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
WARNING 11-20 10:07:27 config.py:428] gguf quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 11-20 10:07:27 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='checkpoint/unsloth.Q5_K_M.gguf', speculative_config=None, tokenizer='checkpoint/unsloth.Q5_K_M.gguf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.GGUF, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gguf, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guid

/home/anhhn/training_qwen/venv/lib/python3.10/site-packages/torch/nested/__init__.py:226: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(


INFO 11-20 10:09:44 model_runner.py:1077] Loading model weights took 9.8409 GB
INFO 11-20 10:10:03 worker.py:232] Memory profiling results: total_gpu_memory=31.74GiB initial_memory_usage=10.38GiB peak_torch_memory=11.26GiB memory_usage_post_profile=10.40GiB non_torch_memory=0.56GiB kv_cache_size=0.87GiB gpu_memory_utilization=0.40
INFO 11-20 10:10:04 gpu_executor.py:113] # GPU blocks: 298, # CPU blocks: 1365
INFO 11-20 10:10:04 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 1.16x
INFO 11-20 10:10:07 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-20 10:10:07 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 1

In [25]:
def run(
    tweet: str | list[str],
    model: LLM,
    prompter: Prompter = prompter,
    temperature: float = 0.1,
    max_tokens: int = 128,
    **kwargs
) -> list[str]:

    if isinstance(tweet, str):
        tweet = [tweet]

    messages: list[list[dict[str, str]]] = []
    for tweet_ in tweet:
        message = [
            {"role": "system", "content": prompter.system_prompt},
            {"role": "user", "content": prompter.prompt_template.format(tweet=tweet_)}
        ]
        messages.append(message)
    
    print(len(messages))

    sampling_params = SamplingParams(
        temperature=temperature,
        max_tokens=max_tokens,
        top_k=kwargs.get("top_k", 40),
        top_p=kwargs.get("top_p", 1.0),
        repetition_penalty=kwargs.get("repetition_penalty", 1.0),
        guided_decoding=GuidedDecodingParams(choice=["calling", "not_calling"])
    )

    outputs = model.chat(
        messages=messages, 
        sampling_params=sampling_params,
        use_tqdm=True,
    )

    outputs = [output.outputs[0].text.strip() for output in outputs]
    return outputs

In [26]:
df = pd.read_excel("data/data_infer_100sample.xlsx")

In [27]:
tweets = df["full_text"].tolist()

In [ ]:
predicts = []
batch_size = 32
for i in range(0, len(tweets), batch_size):
    batch_tweet = tweets[i: i + batch_size]
    batch_predict = run(tweet=batch_tweet, model=llm)
    predicts.extend(batch_predict)

  0%|          | 0/4 [00:00<?, ?it/s]

32


 25%|██▌       | 1/4 [00:44<02:12, 44.31s/it]

32


 50%|█████     | 2/4 [01:28<01:28, 44.22s/it]

32


 75%|███████▌  | 3/4 [02:14<00:44, 44.92s/it]

4


100%|██████████| 4/4 [02:19<00:00, 34.95s/it]


In [29]:
predicts

['not_calling',
 'not_calling',
 'not_calling',
 'calling',
 'calling',
 'not_calling',
 'calling',
 'not_calling',
 'not_calling',
 'calling',
 'calling',
 'calling',
 'not_calling',
 'calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'calling',
 'calling',
 'calling',
 'not_calling',
 'not_calling',
 'calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'calling',
 'calling',
 'calling',
 'not_calling',
 'calling',
 'calling',
 'not_calling',
 'calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'calling',
 'not_calling',
 'calling',
 'calling',
 'calling',
 'calling',
 'not_calling',
 'calling',
 'calling',
 'not_calling',
 'not_calling',
 'calling',
 'not_calling',
 'calling',
 'not_calling',
 'calling',
 'not_calling',
 'not_calling',
 'calling',
 'calling',
 'not_calling',
 'calling',
 'not_calling',
 'not_calling',
 'calling',
 'calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'not_calling',
 'not_ca

In [2]:
!pip list

Package                           Version
--------------------------------- -------------
accelerate                        1.1.1
aiohappyeyeballs                  2.4.3
aiohttp                           3.11.6
aiosignal                         1.3.1
annotated-types                   0.7.0
anyio                             4.6.2.post1
appdirs                           1.4.4
asttokens                         2.4.1
async-timeout                     5.0.1
attrs                             24.2.0
bitsandbytes                      0.44.1
certifi                           2024.8.30
charset-normalizer                3.4.0
click                             8.1.7
cloudpickle                       3.1.0
comm                              0.2.2
compressed-tensors                0.8.0
datasets                          3.1.0
debugpy                           1.8.8
decorator                         5.1.1
dill                              0.3.8
diskcache                         5.6.3
distro           

In [ ]:
!python test_pretraining.py

model-00001-of-00008.safetensors:   0%|            | 10.5M/3.89G [00:00<?, ?B/s]
model-00001-of-00008.safetensors:   1%|   | 21.0M/3.89G [00:21<2:14:10, 480kB/s]
model-00001-of-00008.safetensors:   1%|   | 21.0M/3.89G [00:39<2:14:10, 480kB/s]
model-00001-of-00008.safetensors:   1%|   | 31.5M/3.89G [00:44<2:16:05, 472kB/s]
model-00001-of-00008.safetensors:   1%|   | 31.5M/3.89G [00:59<2:16:05, 472kB/s]
model-00001-of-00008.safetensors:   1%|   | 41.9M/3.89G [01:05<2:11:52, 486kB/s]
model-00001-of-00008.safetensors:   1%|   | 41.9M/3.89G [01:19<2:11:52, 486kB/s]
model-00001-of-00008.safetensors:   1%|   | 52.4M/3.89G [01:28<2:16:20, 469kB/s]
model-00001-of-00008.safetensors:   1%|   | 52.4M/3.89G [01:39<2:16:20, 469kB/s]
model-00001-of-00008.safetensors:   2%|   | 62.9M/3.89G [01:53<2:21:18, 451kB/s]
model-00001-of-00008.safetensors:   2%|   | 62.9M/3.89G [02:09<2:21:18, 451kB/s]
model-00001-of-00008.safetensors:   2%|   | 73.4M/3.89G [02:17<2:22:12, 447kB/s]
model-00001-of-00008.safeten